问题：

1. .o文件是没办法运行的，链接时才将变量所声明的符号和定义关联（C++ 必知必会（二/1）8:34， 元宝）

2. 我明白了，是不是int、*、[]/[10]、()，在声明与定义中，本质上都是声明符。“int a[3]”实际上应该理解为：“int  [3] a”的形式；“int *a”应理解为：“int  * a”；二者结合“int *a[3]”，实际上是“int * [3] a”。所以，“int (*array_ptr)[4];”实际上是“int [4] (*array_ptr);”，通过括号改变了优先级，说明array_ptr首先是个指针，然后它指向长度为4的数组，数组里面的元素类型为int。

   specifier declarator

3. 引用

4. 表达式运算的结果（返回值，所代表的值）和作用，结果分两个属性：类型和值类别，值类别分左右值

5. 底层/顶层cast

6. 全局作用域，::

7. 命名空间作用域 ns-name :: member-name

   using namespace

8. 变量存储期 static

9. 预处理，编译，汇编，链接

10. c++直接构造

# H1 项目的构成和构建

## 项目元素及基本概念

1. 源代码文件

    **源代码文件**：用编程语言（如C++，Java，Python等）写的人类可读的指令文本文件

    - **头文件（`*.hpp`）**
  
        声明函数签名、类结构（告诉编译器“某个东西存在”），但不含具体实现。
      
    - **源文件（`*.cpp`）**
  
        定义（具体实现）函数、类成员函数、全局变量等，实现程序逻辑
  
        当源文件需要用到头文件的声明时，用`#include`包含进来。

2. 编译与链接

    - **编译（Compile）**

        - 将一个源文件与它用得到的头文件，翻译成计算机CPU能直接理解和执行的机器语言（一堆0和1），生成一个目标文件（.o文件）。

        - 执行这个翻译工作的程序叫 **编译器 (Compiler）**，如`g++` on Linux。

        - **编译单元**：一个.cpp文件，以及它包含的头文件。
     
        -  在Linux系统中，可以通过命令`g++ -c source_file.cpp`，编译`source_file.cpp`，生成目标文件`source_file.o`（可通过`-o`参数指定生成目标文件名称）。
  
    - **链接（Linking）**
  
        - 通过 **链接器（Linking）**，如`ld` on Linux，把 **多个编译好的目标文件（.o）** 和需要的 **库文件（.a、.so，可以理解为多个.o文件的集合）**“缝合”在一起，解决它们之间的相互引用（如，main.o里调用了math.o里的函数，或者调用了libm.so里的函数），最终生成一个完整的、可以直接运行的程序文件，即 **可执行文件（.out、.app）**。
  
        - **可执行文件（.out、.app）**：程序存储在磁盘上的形式。
      
            **程序** 是完成特定任务的逻辑抽象（算法+数据结构），它以 **可执行文件形式（静态物理状态）** 存储在磁盘上，当操作系统加载该可执行文件到内存，并为其分配执行环境（内存空间、文件描述符等）后，这个运行实例就成为 **进程（动态执行状态）**。
      
        - 在Linux系统中，可以直接通过`g++`进行链接（`g++`实际上是一个前端，它会调用`ld`来完成链接过程），如：
     
            `g++ main.o another.o -o myprogram`，该命令将`main.o`和`another.o`两个目标文件链接起来，生成名为`myprogram`的可执行文件（若不指定`-o`参数，则默认输出名为`a.out`）。
     
            由于g++本质上是个编译器，所以也可以直接从源文件开始，直接编译加链接到可执行文件，如`g++ main.cpp another.cpp -o myprogram`。

3. 库

    **库(Library)**：一堆预先编写好、编译好的、可以直接调用的代码集合（理解为多个.o文件的集合），提供常用的功能，避免你“重复造轮子”（实现代码复用）

    1. 静态库和动态库
  
       - **静态库（`*.a`）**

            在链接阶段，会被完整地复制到最终生成的可执行文件中。
      
            只引用静态库：
      
            **优点**：可执行文件独立运行，不依赖外部库文件
      
            **缺点**：可执行文件体积大；多个进程使用相同库时，内存中有多份拷贝；库更新需要重新编译链接整个可执行文件。

       - **动态库（`.so`）**
      
            在程序运行时才被加载到内存中。可执行文件里只记录需要哪个库以及库里的哪些函数。
      
            动态库不仅仅是.o文件的集合，它是经过了部分链接的特殊可执行文件（所以才能加载到内存运行）。
      
            只使用动态库：
      
            **优点**：可执行文件体积小；多个进程可共享内存中的同一份库代码，节省内存；库更新方便（替换文件即可），无需重新编译链接可执行文件（需注意接口兼容性）。

            **缺点**：程序运行时必须能找到对应的库文件，否则会出错（“找不到DLL”）。
  
    1. 标准库、第三方库和自己写的库
  
| 类型           | 标准库                                   | 第三方库                                       | 自己写的库                       |
|----------------|----------------------------------------|---------------------------------------------|--------------------------------|
| **谁提供的**    | 语言官方（C++标准委员会）                | 其他开发者/公司（如Google的protobuf）          | 你自己                          |
| **包含什么**    | 基础功能（字符串处理、文件操作等）        | 特定领域功能（网络请求、图形绘制等）            | 项目中重复使用的模块             |
| **如何引用**    | `#include <iostream>`                   | `#include <openssl/md5.h>`                   | `#include "mylib/utils.h"`     |
| **是否需要额外操作** | 直接使用（编译器自带）                  | 需下载安装并配置链接                          | 只需正确包含头文件               |
| **例子**       | `vector`, `string`, `fstream`           | Boost, OpenCV, Qt                            | 自己封装的日志工具、数学工具      |


## 一个常见的C++项目文件结构

```
project_root/
│
├── bin/           # 存放编译生成的可执行文件
├── build/         # 存放编译过程中的中间文件（如CMake的构建文件）
├── docs/          # 项目文档
├── include/       # 公共头文件（通常按模块组织）
│   └── project_name/ # 项目相关的公共头文件，避免命名冲突
├── lib/           # 存放生成的库文件（静态库或动态库）
├── src/           # 源代码文件
│   ├── module1/   # 模块1的源文件
│   ├── module2/   # 模块2的源文件
│   └── main.cpp   # 主程序入口
├── tests/         # 单元测试和集成测试
├── third_party/   # 第三方依赖库
├── CMakeLists.txt # CMake构建配置文件
├── .gitignore     # Git忽略规则
└── README.md      # 项目说明文件
```

1. `bin/` - 可执行文件目录
- **作用**：存放编译后生成的可执行程序
- **内容**：
  - 主程序可执行文件
  - 工具程序
  - 测试程序
- **最佳实践**：
  - 通常添加到 `.gitignore` 中，不入版本控制
  - 包含 `debug/` 和 `release/` 子目录以区分构建类型

2. `build/` - 构建中间文件目录
- **作用**：存放编译过程中的中间文件
- **内容**：
  - CMake生成的文件
  - 编译器目标文件（`.o/.obj`）
  - 编译日志
- **最佳实践**：
  - **永远不入版本控制**
  - 使用不同的子目录区分不同配置（如 `build/debug`, `build/release`）

3. `docs/` - 项目文档目录
- **作用**：存放项目文档
- **内容**：
  - 设计文档（.md, .pdf）
  - API文档（Doxygen/Sphinx生成）
  - 用户手册
  - 开发者指南
- **最佳实践**：
  - 使用子目录分类（如 `api/`, `design/`, `tutorials/`）
  - 文档与代码同步更新

4. `include/` - 公共头文件目录
- **作用**：存放项目中所有模块的公共头文件
- **结构**：
```
include/
└── project_name/ # 项目命名空间目录，防止冲突
├── module1/ # 模块1的公共头文件
│ ├── ClassA.hpp
│ └── ClassB.hpp
├── module2/ # 模块2的公共头文件
└── common.hpp # 全局公共头文件
```
- **最佳实践**：
- 头文件使用 `.hpp` 扩展名
- 实现与声明分离（实现放在 `src/`）
- 每个头文件使用 `#pragma once` 或 include guard

5. `lib/` - 库文件目录
- **作用**：存放编译生成的库文件
- **内容**：
- 静态库（`.a/.lib`）
- 动态库（`.so/.dll`）
- 导入库（`.dll.a/.lib`）
- **最佳实践**：
- 区分调试和发布版本（`lib/debug`, `lib/release`）
- 按平台组织（`lib/linux_x64`, `lib/windows_x64`）
- 通常不入版本控制

6. `src/` - 源代码目录
- **作用**：存放所有源文件实现
- **结构**：
```
src/
├── module1/ # 模块1的实现
│ ├── ClassA.cpp
│ └── ClassB.cpp
├── module2/ # 模块2的实现
│ ├── ClassC.cpp
│ └── ClassD.cpp
└── main.cpp # 程序入口
```
- **最佳实践**：
- 模块目录结构与 `include/` 对应
- 测试文件 **不** 放入此目录
- 使用 `.cpp` 扩展名

7. `tests/` - 测试目录
- **作用**：存放所有单元测试和集成测试
- **结构**：
```
tests/
├── unit/ # 单元测试
│ ├── module1/ # 模块1的测试
│ └── module2/ # 模块2的测试
├── integration/ # 集成测试
└── test_main.cpp # 测试入口
```
- **最佳实践**：
- 使用测试框架（如 Google Test, Catch2）
- 测试文件与源码文件一对一对应（如 `ClassA_test.cpp` 对应 `ClassA.cpp`）
- 包含模拟对象（mocks）用于依赖隔离

8. `third_party/` - 第三方依赖目录
- **作用**：存放项目依赖的第三方库
- **结构**：
```
third_party/
├── boost/ # Boost库
├── spdlog/ # spdlog库
└── CMakeLists.txt # 第三方库的集成配置
```
- **最佳实践**：
- 使用 git submodule 管理
- 或使用包管理器（如 vcpkg, Conan）
- 为每个库提供集成配置

9. 根目录文件
- **`CMakeLists.txt`** - CMake构建系统主配置文件
- **`README.md`** - 项目说明文档
- **`.gitignore`** - Git忽略规则（应包含 `build/`, `bin/`, `lib/`）

## CMake

### Make（Makefile）

在一个大型软件项目中存在问题：

1. 海量的源文件、复杂的头文件关系网、依赖众多外部库：

    - 项目被拆分成成百上千甚至上万个`.cpp/.c`文件，每个文件负责一小块功能。

    - 每个`.cpp`文件通常`#include`多个`.h`文件，声明它需要用到的外部函数、类、变量；头文件之间也会互相`#include`。
  
    - 项目会依赖很多第三方库

2. 编译过程漫长、链接过程复杂且关键：

    - 编译每个`.cpp`文件成一个`.o`文件是独立且耗时的（尤其是大型项目）。
  
    - 修改一个文件后重新编译，有时只需要重新编译那个文件本身和依赖它的少数文件（增量编译），但有时需要全部重来（清理后完全编译）。

    - 链接器需要把成千上万的`.o`文件拼起来，并把对库（静态库直接复制，动态库记录引用）的调用都正确连接上。
  
    - 任何符号（函数名、变量名）找不到（未定义引用）或者重复定义都会导致链接失败。这是大型项目中常见的错误来源。
  
手动敲命令去编译每个文件、指定所有依赖、链接所有东西，在大型项目中是完全不可行的，效率极低且极易出错。所以，我们需要有一个**构建工具**来自动化地帮助我们构建（编译和链接）软件项目。在Linux系统下，这通常是`Make`。

对于一般的构建工具，需要你编写**构建文件**，来选择**调用哪个编译器编译每个源文件**、处理**源代码文件间的依赖关系**、指定**构建规则和顺序**、管理**增量编译过程**等。比如，`Make`的脚本文件`Makefile`。

### CMake

在不同的操作系统（Windows，macOS，Linux），以及在其下使用不同的编译器（Visual Studio，GCC，Clang），所适配的构建工具不尽相同，其对应构建文件的语法也千差万别。

`CMake`正是解决了这个问题，其核心工作：

1. 写一个“`CMakeLists.txt`”（配置说明书）： 用相对简单、统一的语法（CMake自己的语言），在文件`CMakeLists.txt`里描述项目，主要内容大致包括：

    - 项目名字

    - 需要的源代码文件（`.c`，`.cpp`）

    - 需要链接的库（比如数学库`libm`、图形库`OpenGL`）

    - 对应库的头文件（`.h`，`.hpp`）

    - 生成一个可执行程序或是库

2. `CMake`读取“`CMakeLists.txt`”并生成真正的“构建文件”：

    `CMake`会自动检测，构建项目时，所处的环境，包括操作系统、编译器、构建工具类型等；然后自动生成一套适合该特定环境的“真正的构建文件”。当然，你也可以在`CMakeLists.txt`中，显示指定该项目构建所处的环境。

   比如，在Windows + Visual Studio上，它生成`.sln`和`.vcxproj`配置文件；在Linux/macOS + Make上，它生成`Makefile`脚本文件。

`CMake`的价值体现在——它提供了一个统一的抽象层来描述项目结构和依赖 (`CMakeLists.txt`)，然后由它去自动适应不同平台和工具链的复杂性，生成本地化的高效构建脚本。

### 基本使用流程

1. 在项目根目录（有`CMakeLists.txt`的地方）新建一个空文件夹（通常叫`build`）。

2. 进入`build`文件夹。

3. 运行命令`cmake ..`(这告诉`CMake`去上一层目录找`CMakeLists.txt`，并在当前`build`目录生成构建文件)。

4. 运行`cmake --build .`（调用本地构建工具开始编译），在Linux下相当于`make`，在Windows下可能需要用Visual Studio打开生成的`.sln`或运行`cmake --build .`）。

### CMake基本语法

# C++基本语法

## 变量、数组与函数

字面量

声明 specifier declarator

## 表达式